In [1]:
import pandas as pd
import pickle

In [2]:
file_path = '../data/airoport_data.pkl'
with open(file_path, 'rb') as file:
    countries = pickle.load(file)

file_path = '../data/countries_continents.pkl'
with open(file_path, 'rb') as file:
    continents = pickle.load(file)

In [3]:
sirena = pd.read_csv('../data/Sirena-export-fixed.csv', index_col='id')

In [4]:
sirena.head()

,PaxName,BirthDate,DepartDate,DepartTime,ArrivalDate,ArrivalTime,FlightCode,From,Dest,Code e-Ticket,TravelDoc,Seat,Meal,Class,Baggage,PaxAdditionalInfo,Loyality,AgentInfo
id,,,,,,,,,,,,,,,,,,
0,ОЗЕРОВ ИЛЬДАР ДАНИИЛОВИЧ,1999-05-15,2017-05-30,00:05,2017-05-30,08:05,SU1306NO,SVO,OVB,ZBQSPY7360415302044672,9375 053270,NaN,NaN,J,JGRPGN0PC,S,FF#SU 38116280,Go2See
1,КОЛОСОВ САМИР ТАМЕРЛАНОВИЧ,NaN,2017-12-27,02:15,2017-12-27,04:40,SU1323NO,MMK,SVO,MHPBBX7398421117936516,2244 645520,NaN,KSML,Y,YRSTUQ,9,FF#FB 284903754,NaN
2,ИГНАТОВА СНЕЖАНА КОНСТАНТИНОВНА,NaN,2017-09-19,06:40,2017-09-19,07:45,SU1481NO,KJA,SVO,REDLVB5174973140468001,8115 961316,NaN,NaN,Y,YSTNJL,F,NaN,KupiBilet
3,ЖАРОВ ПЛАТОН АЛЬБЕРТОВИЧ,1999-05-02,2017-03-18,22:10,2017-03-19,01:05,SU1180NO,SVO,VOG,NSJNGQ5274206497242737,98 6865148,NaN,NaN,J,JFLXLS0PC,#,FF#FB 884556993,Travelgenio
4,НИКОЛЬСКИЙ НИКОЛАЙ ИГОРЕВИЧ,1990-12-26,2017-03-18,22:10,2017-03-19,01:05,SU1180NO,SVO,VOG,VWNYGF6247422701565929,4396 926588,NaN,NaN,Y,YFLXPG,NaN,FF#SU 183142068,OZON.travel


### Переведем именна колонок в snake_case

In [6]:
sirena.columns = [
    'pax_name',
    'birth_date',
    'depart_date',
    'depart_time',
    'arrival_date',
    'arrival_time',
    'flight_code',
    'from',
    'dest',
    'code_e_ticket',
    'travel_doc',
    'seat',
    'meal',
    'class',
    'baggage',
    'pax_additional_info',
    'loyality',
    'agent_info'
]

In [7]:
sirena.head()

,pax_name,birth_date,depart_date,depart_time,arrival_date,arrival_time,flight_code,from,dest,code_e_ticket,travel_doc,seat,meal,class,baggage,pax_additional_info,loyality,agent_info
id,,,,,,,,,,,,,,,,,,
0,ОЗЕРОВ ИЛЬДАР ДАНИИЛОВИЧ,1999-05-15,2017-05-30,00:05,2017-05-30,08:05,SU1306NO,SVO,OVB,ZBQSPY7360415302044672,9375 053270,NaN,NaN,J,JGRPGN0PC,S,FF#SU 38116280,Go2See
1,КОЛОСОВ САМИР ТАМЕРЛАНОВИЧ,NaN,2017-12-27,02:15,2017-12-27,04:40,SU1323NO,MMK,SVO,MHPBBX7398421117936516,2244 645520,NaN,KSML,Y,YRSTUQ,9,FF#FB 284903754,NaN
2,ИГНАТОВА СНЕЖАНА КОНСТАНТИНОВНА,NaN,2017-09-19,06:40,2017-09-19,07:45,SU1481NO,KJA,SVO,REDLVB5174973140468001,8115 961316,NaN,NaN,Y,YSTNJL,F,NaN,KupiBilet
3,ЖАРОВ ПЛАТОН АЛЬБЕРТОВИЧ,1999-05-02,2017-03-18,22:10,2017-03-19,01:05,SU1180NO,SVO,VOG,NSJNGQ5274206497242737,98 6865148,NaN,NaN,J,JFLXLS0PC,#,FF#FB 884556993,Travelgenio
4,НИКОЛЬСКИЙ НИКОЛАЙ ИГОРЕВИЧ,1990-12-26,2017-03-18,22:10,2017-03-19,01:05,SU1180NO,SVO,VOG,VWNYGF6247422701565929,4396 926588,NaN,NaN,Y,YFLXPG,NaN,FF#SU 183142068,OZON.travel


### Определим страны по коду аэропорта

In [9]:
sirena['departure_country'] = sirena['from'].map(countries)
sirena['arrival_country']   = sirena['dest'].map(countries)

### Определим континенты по странам

In [11]:
sirena['departure_continent'] = sirena['departure_country'].map(continents).fillna('Asia')
sirena['arrival_continent']   = sirena['arrival_country'].map(continents).fillna('Asia')

### Создаем таблицу данных

In [13]:
def count_suspicious_flights(df):
    prev_airport = None
    susp_flights = cross_country = susp_cross_country = long_flights = 0
    for src_airport, dst_airport, src_country, dst_country, src_continent, dst_continent in df.values:
        if prev_airport != None and src_airport.lower() != prev_airport.lower():
            susp_flights += 1
            if src_country.lower() != dst_country.lower():
                susp_cross_country += 1
        prev_airport = dst_airport
        
        if src_country.lower() != dst_country.lower():
            cross_country += 1
        if src_continent.lower() != dst_continent.lower():
            long_flights += 1
    return susp_flights, cross_country, susp_cross_country, long_flights

In [14]:
suspicious_flights_df = pd.DataFrame(columns=['name', 'flights_count', 'suspicious_flights', 
                                              'cross_country', 'suspicious_cross_country', 'long_flights'])

for i, name in enumerate(sirena.pax_name.unique()):
    df = sirena[sirena['pax_name'] == name].sort_values(by='depart_date').reset_index(drop=True)
    flights_count = len(df)
    
    
    susp_flights, cross_country, susp_cross_country, long_flights = count_suspicious_flights(df[['from', 'dest', 
                                                                                                 'departure_country', 'arrival_country', 
                                                                                                 'departure_continent', 'arrival_continent']])
    
    row_to_add = [name, flights_count, susp_flights, cross_country, susp_cross_country, long_flights]
    suspicious_flights_df.loc[i] = row_to_add

### Вычислим процент подозрительных и международных рейсов

In [16]:
suspicious_flights_df['percent_suspicious_flights'] = suspicious_flights_df['suspicious_flights'] / suspicious_flights_df['flights_count']

In [17]:
suspicious_flights_df['percent_cross_country'] = suspicious_flights_df['cross_country'] / suspicious_flights_df['flights_count']

In [18]:
suspicious_flights_df

,name,flights_count,suspicious_flights,cross_country,suspicious_cross_country,long_flights,percent_suspicious_flights,percent_cross_country
0,ОЗЕРОВ ИЛЬДАР ДАНИИЛОВИЧ,1,0,0,0,0,0.000000,0.0
1,КОЛОСОВ САМИР ТАМЕРЛАНОВИЧ,7,3,0,0,0,0.428571,0.0
2,ИГНАТОВА СНЕЖАНА КОНСТАНТИНОВНА,5,1,0,0,0,0.200000,0.0
3,ЖАРОВ ПЛАТОН АЛЬБЕРТОВИЧ,5,0,0,0,0,0.000000,0.0
4,НИКОЛЬСКИЙ НИКОЛАЙ ИГОРЕВИЧ,6,2,0,0,0,0.333333,0.0
...,...,...,...,...,...,...,...,...
72730,МАСЛОВА ЯСМИНА МАКАРЬЕВНА,1,0,0,0,0,0.000000,0.0
72731,БОЧАРОВА МИЛЕНА ДЕМИДОВНА,1,0,0,0,0,0.000000,0.0
72732,ЯКОВЛЕВ АРТЁМ АНАТОЛЬЕВИЧ,1,0,0,0,0,0.000000,0.0
72733,ОЗЕРОВ ГЕОРГИЙ СЕРГЕЕВИЧ,1,0,0,0,0,0.000000,0.0


In [19]:
suspicious_flights_df.to_csv('../data/ready/Sirena_susp_flights.csv', index_label='id')